In [1]:
import os
import json
import csv
import nbformat
import pandas as pd
import numpy as np
from datetime import datetime
from markdown import markdown
from casstools.cass_client import CassClient
from casstools.notebook_tools import NotebookFile
import logging

OK = "\U00002705"
QUESTION = "\U00002753"
CANCEL = "\U0000274C"


logging.basicConfig(level=logging.INFO)

In [2]:
client = CassClient()

In [3]:
def get_templates(filespec="./templates/*.ipynb"):
    from glob import glob
    return sorted(glob(filespec))

In [4]:
def load_template_header_cell(filespec):
    import uuid
    with open(filespec) as f:
        notebook = nbformat.read(f,  as_version=4)
        markdown = [m for m in notebook.cells if m.cell_type == "markdown"]
        template_cell = markdown[0]
        template_cell.metadata['cass'] = {'cell_type': "grading_header"}
        template_cell.id = f"{uuid.uuid4()}"
    return template_cell


#load_template_header_cell("./templates/Homework.ipynb")

In [5]:
def initialize_directories(basepath="."):
    directories = ['blackboard', 'fetched', 'templates']
    for d in directories:
        pathspec = os.path.join(basepath, d)
        if not os.path.exists(pathspec):
            os.makedirs(pathspec)

In [6]:
def remove_time_offset(time_string_iso_with_offset: str, timezone: str) -> str:
    from zoneinfo import ZoneInfo
    dt = datetime.fromisoformat(time_string_iso_with_offset).astimezone(ZoneInfo("America/New_York"))
    dts = dt.strftime("%Y-%m-%d %H:%M")
    return dts

In [7]:
def make_clickable(url, name, section=""):
    if url == "":
        return name
    else:
        if section == "":
            return '<a href="{}" rel="noopener noreferrer">{}</a>'.format(url, name)
        else:
            return '<a href="{}#{}" rel="noopener noreferrer">{}</a>'.format(url, section, name)


In [8]:
def clear_assignments_folder(course_key, assignment_name):
    import shutil
    folder = get_assignments_folder(course_key, assignment_name)
    shutil.rmtree(folder)
    return folder

In [9]:
def get_assignments_folder(course_key, assignment_name):
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    return folder


In [10]:
def load_submissions_file(filespec):
    if os.path.exists(filespec):
        with open(filespec, "r") as f:
            submissions = json.load(f)
    else:
        submissions = []
        
    return submissions
        
def save_submissions_file(filespec, submissions):
    with open(filespec, "w") as f:
        buff = json.dumps(submissions)
        f.write(buff)


In [11]:
def get_assignments_folder_url(course_key, assignment_name):
    ''' 
    makes assumption cassgrader is in library folder
    '''
    me = client.whoami()
    base = f"https://v2hub.ischool.syr.edu/user/{me['name']}/lab/workspaces/auto-T/tree/library/cassgrader"
    assignments = client.get_course_assignments(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    folder = f"/fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    url = base + folder
    return url

In [12]:
def display_submissions(course_key, assignment_name) -> pd.DataFrame:
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    submissions_file = f"{folder}/submissions.json"
    submissions = load_submissions_file(submissions_file)
    if submissions != []:
        df = pd.DataFrame(submissions)
        df['view_submission'] = df.apply(lambda row: make_clickable(row['notebookfile'], row['student'] ), axis=1)
        return df.sort_values("view_submission")
    else:
        df = pd.DataFrame()
        return df 
    


In [13]:
def build_row(student_name,info, assignment_duedate):
    row = { 'student' : student_name, 'status' : "NONE", 'template_file': "" }
    if info['exists']:
        row['submitted'] = True
        row['etag'] = info['etag']
        row['last_modified'] = info['last_modified']
        row['submitted_on'] = remove_time_offset(info['last_modified'], "America/New_York")
        row['object_on_s3'] = info['object']
        row['late'] = datetime.fromisoformat(row['submitted_on']) > datetime.fromisoformat(assignment_duedate)
    else:
        row['submitted'] = False
        row['etag'] = ""
        row['last_modified'] = ""
        row['submitted_on'] = ""
        row['object_on_s3'] = ""
        row['late'] = False
    return row

In [14]:
def inject_template(notebook, template_cell):
    first_cell = notebook['cells'][0]
    cass = first_cell['metadata'].get('cass',None)
    if cass is None or cass != {'cell_type': "grading_header"}:
        notebook['cells'].insert(0,template_cell)

    return notebook
    

In [15]:
def download_submission_file(course_key, assignment_name, student_name, file_name, folder, template_file):
    template_cell = load_template_header_cell(template_file)
    notebook = client.get_assignment_submission(course_key, assignment_name, student_name, file_name)  
    notebook = inject_template(notebook, template_cell)
    #nbformat.validate(notebook)   
    notebookfile = f"{folder}/{student_name}.ipynb"
    with open(notebookfile, "w") as f:
        f.write(json.dumps(notebook))
        
    return notebookfile

# folder = f"./fetched/tst101-spring2024/mafudge@syr.edu/A/A2.ipynb"
# download_submission_file("tst101-spring2024", "A2.ipynb" , "mafudge@syr.edu", "A2.ipynb", folder, "./templates/Homework.ipynb")


In [16]:
def fetch_assignments(course_key, assignment_name, template_file) -> pd.DataFrame:
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster[ (roster['instructor']==me['name']) | (roster['grader']==me['name']) ]['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    os.makedirs(folder, exist_ok=True)
    
    #Load Submissions database
    submissions_file = f"{folder}/submissions.json"    
    submissions = load_submissions_file(submissions_file)
    
    # PROCESS FOR EACH STUDENT 
    for student_name in student_list:
        
        # FETCH CLOUD INFO
        info = client.submission_info(course_key, assignment_name, student_name, assignment_name)
        row = build_row(student_name, info, assignment_dict['due_date'])
        
        # WHEN STUDENT EXISTS...
        submission_rows = [ s for s in submissions if s['student'] == row['student']]
        if len(submission_rows) >0:
            submission_row = submission_rows[0]
            submission_row_index = submissions.index(submission_row)
            
            # DOES THE EXISTING STUDENT HAVE A SUBMISSION?
            if submission_row['submitted']:
                
                # DO THE ETAGS MATCH? THEN ONLY DOWNLOAD WHEN FILES DOESNT EXIST
                if submission_row['etag'] == row['etag']:

                    submission_row['last_fetch'] = "Same:Skipping"
                    if not os.path.exists(submission_row['notebookfile']):
                        submission_row['notebookfile'] = download_submission_file(course_key, assignment_name, student_name, assignment_name, folder, template_file)
                        submission_row['template_file'] = template_file
                        submissions[submission_row_index] = submission_row
                        logging.debug(f"{submission_row['last_fetch']} => {student_name} has submission and etags match, but file didn't exist, downloading.")
                    else:
                        logging.debug(f"{submission_row['last_fetch']} => {student_name} has submission and etags match, skipping.")
                    submissions[submission_row_index] = submission_row
                
                # ETAGS DO NOT MATCH... RESUBMISSION
                else: 
                    #TODO: updated file... what to do where
                    existingfile = f"{folder}/{student_name}.ipynb"
                    backupfile = f"{folder}/{student_name}-{submission_row['etag']}.ipynb"
                    os.rename(existingfile,backupfile)
                    row['notebookfile'] = download_submission_file(course_key, assignment_name, student_name, assignment_name, folder, template_file)
                    row['template_file'] = template_file
                    row['last_fetch'] = "Newer:Updating"
                    submissions[submission_row_index] = row
                    logging.debug(f"{submission_row['last_fetch']} => {student_name} has a re-submission that is newer.")

            # EXISTING STUDENT DOES NOT HAVE An EXISTING SUBMISSION
            else:
                
                # IS THERE A CLOUD SUBMSSION?
                if row['submitted']:
                    row['notebookfile'] = download_submission_file(course_key, assignment_name, student_name, assignment_name, folder, template_file)
                    row['template_file'] = template_file
                    row['last_fetch'] = "New:Updating"
                    submissions[submission_row_index] = row
                    logging.debug(f"{row['last_fetch']} => {student_name} was updated with cloud submission.")
                                    
                
        # STUDENT DOES NOT EXIST ADD ROW, AND DOWNLOAD FILE (IF EXISTS)
        else:
            row['last_fetch'] = "New:Added"
            if row['submitted']:
                row['notebookfile'] = download_submission_file(course_key, assignment_name, student_name, assignment_name, folder, template_file)
                row['template_file'] = template_file                
            else:
                row['notebookfile'] = ""
            
            logging.debug(f"{row['last_fetch']} => {student_name} not in submission database, adding")
            submissions.append(row)

    save_submissions_file(submissions_file, submissions)

In [17]:
def extract_grade(header_cell_text):
    for line in header_cell_text.split("\n"):
        if line.strip().find("- Your Grade") >=0:
            grade = line.split(":")[-1].strip()
            if grade.isdigit():
                return int(grade)
            else:
                return None

In [18]:
def bb_filename(assignment_name, bbcolumn):
    pos = bbcolumn.find("|")
    if pos>=0:
        bbfile = assignment_name.split(".")[0] + "-" + bbcolumn[pos+1:] + ".csv"
    else:
        bbfile = assignment_name.split(".")[0] + "-" + bbcolumn + ".csv"
    
    return bbfile

#bb_filename("A1.ipynb", "|351235")

In [19]:
def create_bb_file(course_key, assignment_name):
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    bbfilename = bb_filename(assignment_name, str(assignment_dict['lms_column_id']))
    bbfilespec = os.path.join("./blackboard", bbfilename)

    #Load Submissions database
    submissions_file = f"{folder}/submissions.json"                               
    submissions = load_submissions_file(submissions_file)

    grades = []
    for student in submissions:
        if student['submitted']:
            header_cell = load_template_header_cell(student['notebookfile'])
            grade = extract_grade(header_cell['source'])
            if grade is None:
                print(f"{student['student']} cannot parse grade. Not included in grade file.")
            elif grade > int(assignment_dict['points']):
                print(f"{student['student']} grade of {grade} is higher than assignment points {assignment_dict['points']}. Not included in grade file.")
            else:
                grades.append({
                    "Username": student['student'].split("@")[0].strip() , 
                    assignment_dict['lms_column_id'] : grade , 
                    "Grading Notes" : "", 
                    "Notes Format" : "HTML", 
                    "Feedback to Learner" : markdown(header_cell['source']),  
                    "Feedback Format" : "HTML"
                })

    if len(grades) >0:
        grades_df =  pd.DataFrame(grades) 
        grades_df.to_csv(bbfilespec,sep=",",header=True,index=False, quoting=csv.QUOTE_NONNUMERIC)

        return len(grades), bbfilespec, grades_df

    else:
        return 0, "None", None


# create_bb_file("tst101-spring2024","A2.ipynb")
    

In [24]:
def assignment_check(file, RUN_CHECK_CODE):
    if file == "":
        return ""
    nb = NotebookFile(file)
    lab_check_cell = [cell for cell in nb.code_cells if cell.source.find("NotebookFile().check_lab()") >= 0]
    if len(lab_check_cell) != 0 and lab_check_cell[0].get('execution_count', None) is not None:
        lab_check_cell_output = lab_check_cell[0]['outputs'][0]['text']
    else:
        return f"Check code was not found in the submission, or not executed by the student."

    new_output = ""
    for row in lab_check_cell_output.split("\n"):
        items = row.split()
        if len(items) > 0:
            you_code = items[1] if items[0] == CANCEL else items[0]
            if you_code in ["1.1", "1.2", "1.3", "1.4", "1.5", "1.6", "1.7", '2.1', '2.2', '2.3', '2.4', '2.5', '2.6', '2.7']:
                row = row.replace(you_code, make_clickable(file, you_code, f"{you_code}-You-Code"))
        new_output += row + "\n"
    return "<code>" + new_output.strip() + "</code>"

In [ ]:
def extract_cell_contents_if_exists(notebookfile, cell_label_type):
    if notebookfile == "":
        return ""
    nb = NotebookFile(notebookfile)
    cells = nb.markdown_cells_of_type(cell_label_type)
    if len(cells) == 0:
        return ""
    return cells[0].get("source", "")


In [21]:
def display_lab_submissions(course_key, assignment_name) -> pd.DataFrame:
    RUN_CHECK_CODE = "from casstools.notebook_tools import NotebookFile\nNotebookFile().check_lab()"
    me = client.whoami()
    assignments = client.get_course_assignments(course_key)
    roster = client.get_course_roster(course_key)
    assignment_dict = assignments[ assignments['assignment_name'] == assignment_name].to_dict(orient="records")[0]
    student_list = roster['student'].tolist()
    folder = f"./fetched/{course_key}/{me['name']}/{assignment_dict['unit_name']}/{assignment_dict['assignment_name']}"
    submissions_file = f"{folder}/submissions.json"
    submissions = load_submissions_file(submissions_file)
    if submissions != []:
        df = pd.DataFrame(submissions)
        df['view_submission'] = df.apply(lambda row: make_clickable(row['notebookfile'], row['student']), axis=1)
        df['lab_check'] = df.apply(lambda row: assignment_check(row['notebookfile'], RUN_CHECK_CODE), axis=1)
        df['comfort'] = df.apply(lambda row: extract_cell_contents_if_exists(row['notebookfile'], "comfort_cell"), axis=1)
        df['questions'] = df.apply(lambda row: extract_cell_contents_if_exists(row['notebookfile'], "question_cell"), axis=1)
        return df.sort_values("view_submission")
    else:
        df = pd.DataFrame()
        return df 

In [22]:
if __name__ == '__main__':
    file = "./fetched/ist256-spring2024/mafudge@syr.edu/01-Intro/LAB-Intro.ipynb/bsaltman@syr.edu.ipynb"
    nb = NotebookFile(file)
    RUN_CHECK_CODE = "from casstools.notebook_tools import NotebookFile\nNotebookFile().check_lab()"
    output = assignment_check(file, RUN_CHECK_CODE)
    comfort = nb.markdown_cells_of_type("comfort_cell")[0]['source']
    comments = nb.markdown_cells_of_type("question_cell")[0]['source']
    pd.DataFrame([ {
        'user': 'testing',
        'assignment check': output,
        'comfort': comfort,
        'comments': comments
    }]).style.set_properties(**{'text-align': 'left'})

In [23]:
# submission_files = f"{self.__fetch_folder}/{self.env.instructor_netid}/{lesson}/{assignment_file}/*.ipynb"
#     grades = []
#     for filespec in glob.glob(submission_files):
#         logging.debug(f"create_bb_file() => Reading:{filespec}")
#         grade_cell = self.get_grading_cell(filespec)
#         grade = self.get_grade(grade_cell)
#         if grade != None:
#             basename = filespec.split("/")[-1]
#             student_netid = basename.replace(".ipynb","")
#             grades.append({
#                 "Username": student_netid.replace("@syr.edu","") , 
#                 bbcolumn : grade , 
#                 "Grading Notes" : "", 
#                 "Notes Format" : "HTML", 
#                 "Feedback to Learner" : self.grading_cell_to_html(grade_cell),  
#                 "Feedback Format" : "HTML"
#             })

#     if len(grades) >0:
#         grades_df =  pd.DataFrame(grades) 
#         display(grades_df)
#         grades_df.to_csv(bbfilespec,sep=",",header=True,index=False, quoting=csv.QUOTE_NONNUMERIC)
#         logging.debug(f"Created Blackboard Grades File: {bbfilespec}")
#     else:
#         logging.debug("No Grades to Process")

#     return len(grades), bbfile
